<a href="https://colab.research.google.com/github/sagaruprety/Machine-Learning/blob/master/Bravura_job_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [5]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [5]:
!kaggle datasets download -d kandij/job-recommendation-datasets

 15% 8.00M/52.4M [00:00<00:05, 8.68MB/s]
User cancelled operation


In [ ]:
!unzip \*.zip  && rm *.zip

In [6]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [19]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
combined_jobs = pd.read_csv('Combined_Jobs_Final.csv')
list(combined_jobs)

['Job.ID',
 'Provider',
 'Status',
 'Slug',
 'Title',
 'Position',
 'Company',
 'City',
 'State.Name',
 'State.Code',
 'Address',
 'Latitude',
 'Longitude',
 'Industry',
 'Job.Description',
 'Requirements',
 'Salary',
 'Listing.Start',
 'Listing.End',
 'Employment.Type',
 'Education.Required',
 'Created.At',
 'Updated.At']

In [20]:
combined_jobs.head()

,Job.ID,Provider,Status,Slug,Title,Position,Company,City,State.Name,State.Code,Address,Latitude,Longitude,Industry,Job.Description,Requirements,Salary,Listing.Start,Listing.End,Employment.Type,Education.Required,Created.At,Updated.At
0,111,1,open,palo-alto-ca-tacolicious-server,Server @ Tacolicious,Server,Tacolicious,Palo Alto,California,CA,NaN,37.443346,-122.161170,Food and Beverages,Tacolicious' first Palo Alto store just opened...,NaN,8.00,NaN,NaN,Part-Time,NaN,2013-03-12 02:08:28 UTC,2014-08-16 15:35:36 UTC
1,113,1,open,san-francisco-ca-claude-lane-kitchen-staff-chef,Kitchen Staff/Chef @ Claude Lane,Kitchen Staff/Chef,Claude Lane,San Francisco,California,CA,NaN,37.789830,-122.404268,Food and Beverages,\r\n\r\nNew French Brasserie in S.F. Financia...,NaN,0.00,NaN,NaN,Part-Time,NaN,2013-04-12 08:36:36 UTC,2014-08-16 15:35:36 UTC
2,117,1,open,san-francisco-ca-machka-restaurants-corp-barte...,Bartender @ Machka Restaurants Corp.,Bartender,Machka Restaurants Corp.,San Francisco,California,CA,NaN,37.795597,-122.402963,Food and Beverages,We are a popular Mediterranean wine bar and re...,NaN,11.00,NaN,NaN,Part-Time,NaN,2013-07-16 09:34:10 UTC,2014-08-16 15:35:37 UTC
3,121,1,open,brisbane-ca-teriyaki-house-server,Server @ Teriyaki House,Server,Teriyaki House,Brisbane,California,CA,NaN,37.685073,-122.400275,Food and Beverages,● Serve food/drinks to customers in a profess...,NaN,10.55,NaN,NaN,Part-Time,NaN,2013-09-04 15:40:30 UTC,2014-08-16 15:35:38 UTC
4,127,1,open,los-angeles-ca-rosa-mexicano-sunset-kitchen-st...,Kitchen Staff/Chef @ Rosa Mexicano - Sunset,Kitchen Staff/Chef,Rosa Mexicano - Sunset,Los Angeles,California,CA,NaN,34.073384,-118.460439,Food and Beverages,"Located at the heart of Hollywood, we are one ...",NaN,10.55,NaN,NaN,Part-Time,NaN,2013-07-17 15:26:18 UTC,2014-08-16 15:35:40 UTC


In [10]:
print(combined_jobs.shape)
combined_jobs.isnull().sum()

(84090, 23)


Job.ID                    0
Provider                  0
Status                    0
Slug                      0
Title                     0
Position                  0
Company                2271
City                    135
State.Name              171
State.Code              171
Address               84054
Latitude                  0
Longitude                 0
Industry              83823
Job.Description          56
Requirements          84090
Salary                83861
Listing.Start           683
Listing.End             167
Employment.Type          10
Education.Required      267
Created.At                0
Updated.At                0
dtype: int64

In [21]:
new_cols = list(['Job.ID']+['Slug']+['Title']+['Position']+ ['Company']+['City']+['Employment.Type']+['Education.Required']+['Job.Description'])
final_jobs = combined_jobs[new_cols]
final_jobs.columns = ['Job_Id','Slug', 'Title', 'Position', 'Company','City', 'Empl_type','Edu_req','Job_Description']
final_jobs.head() 

,Job_Id,Slug,Title,Position,Company,City,Empl_type,Edu_req,Job_Description
0,111,palo-alto-ca-tacolicious-server,Server @ Tacolicious,Server,Tacolicious,Palo Alto,Part-Time,NaN,Tacolicious' first Palo Alto store just opened...
1,113,san-francisco-ca-claude-lane-kitchen-staff-chef,Kitchen Staff/Chef @ Claude Lane,Kitchen Staff/Chef,Claude Lane,San Francisco,Part-Time,NaN,\r\n\r\nNew French Brasserie in S.F. Financia...
2,117,san-francisco-ca-machka-restaurants-corp-barte...,Bartender @ Machka Restaurants Corp.,Bartender,Machka Restaurants Corp.,San Francisco,Part-Time,NaN,We are a popular Mediterranean wine bar and re...
3,121,brisbane-ca-teriyaki-house-server,Server @ Teriyaki House,Server,Teriyaki House,Brisbane,Part-Time,NaN,● Serve food/drinks to customers in a profess...
4,127,los-angeles-ca-rosa-mexicano-sunset-kitchen-st...,Kitchen Staff/Chef @ Rosa Mexicano - Sunset,Kitchen Staff/Chef,Rosa Mexicano - Sunset,Los Angeles,Part-Time,NaN,"Located at the heart of Hollywood, we are one ..."


In [12]:
final_jobs.isnull().sum()

Job_Id                0
Slug                  0
Title                 0
Position              0
Company            2271
City                135
Empl_type            10
Edu_req             267
Job_Description      56
dtype: int64

In [23]:
#selecting NaN rows of city
no_city = final_jobs[pd.isnull(final_jobs['City'])]
print(no_city.shape)

#finding names of companies which do not have a city name listed
no_city['Company'].unique()

(0, 9)


array([], dtype=object)

In [25]:
#replacing nan with New York
final_jobs.loc[:,'City'] = final_jobs.loc[:,'City'].fillna('New York')

#Similarly replacing NAN values in employment type with Part-Time
final_jobs.loc[:,'Empl_type'] = final_jobs.loc[:,'Empl_type'].fillna('Part-Time')


In [26]:
#Combining all relevant textual columns to form job-document

final_jobs["Job_Doc"] = final_jobs["Position"].map(str) + " " + final_jobs["Company"] +" "+ final_jobs["City"]+ " "+final_jobs['Empl_type']+" "+final_jobs['Job_Description']
final_jobs['Job_Doc'] = final_jobs['Job_Doc'].str.replace('[^a-zA-Z \n\.]'," ")
final_jobs['Job_Doc'] = final_jobs['Job_Doc'].str.lower()
job_docs = final_jobs[['Job_Id','Job_Doc']]
job_docs = job_docs.dropna()
job_docs.shape

(81766, 2)

In [27]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
#removing stopwords

_job_doc = job_docs['Job_Doc']
from nltk.corpus import stopwords
stop = stopwords.words('english')
_no_stop = _job_doc.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
_no_stop.head()

job_docs['Job_Doc'] = _no_stop
job_docs.head()

,Job_Id,Job_Doc
0,111,server tacolicious palo alto part time tacolic...
1,113,kitchen staff chef claude lane san francisco p...
2,117,bartender machka restaurants corp. san francis...
3,121,server teriyaki house brisbane part time serve...
4,127,kitchen staff chef rosa mexicano sunset los an...


In [29]:
job_docs.to_csv("job_docs.csv", index=True)

In [7]:
#Vectorising the data
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
job_docs = pd.read_csv('job_docs.csv')

job_vecs = tfidf_vectorizer.fit_transform((job_docs['Job_Doc'])) #fitting and transforming the vector

In [31]:
#User historical data from positions viewed is considered to form the query

user_view_history = pd.read_csv('Job_Views.csv')
user_view_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12370 entries, 0 to 12369
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Applicant.ID   12370 non-null  int64  
 1   Job.ID         12370 non-null  int64  
 2   Title          12370 non-null  object 
 3   Position       12370 non-null  object 
 4   Company        11790 non-null  object 
 5   City           12370 non-null  object 
 6   State.Name     12348 non-null  object 
 7   State.Code     12348 non-null  object 
 8   Industry       2199 non-null   object 
 9   View.Start     12370 non-null  object 
 10  View.End       10575 non-null  object 
 11  View.Duration  10575 non-null  float64
 12  Created.At     12370 non-null  object 
 13  Updated.At     12370 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 1.3+ MB


In [32]:
_user_view_relevant = user_view_history[['Applicant.ID', 'Job.ID', 'Position', 'Company','City']]

_user_view_relevant["View_Query"] = _user_view_relevant["Position"].map(str) + "  " + _user_view_relevant["Company"] +"  "+ _user_view_relevant["City"]

_user_view_relevant['View_Query'] = _user_view_relevant['View_Query'].str.replace('[^a-zA-Z \n\.]',"")

_user_view_relevant['View_Query'] = _user_view_relevant['View_Query'].str.lower()

user_view_history = _user_view_relevant[['Applicant.ID','View_Query']]

user_view_history.head()

,Applicant.ID,View_Query
0,10000,cashiers valets needed wallypark newark
1,10000,macys seasonal retail fragrance cashier gard...
2,10001,part time showroom sales cashier grizzly ind...
3,10002,event specialist part time advantage sales m...
4,10002,bonefish kitchen staff bonefish grill green...


In [33]:
#User's current employment position data is also considered to form the query
experience = pd.read_csv('Experience.csv')
experience.head()

,Applicant.ID,Position.Name,Employer.Name,City,State.Name,State.Code,Start.Date,End.Date,Job.Description,Salary,Can.Contact.Employer,Created.At,Updated.At
0,10001,Account Manager / Sales Administration / Quali...,Barcode Resourcing,Bellingham,Washington,WA,2012-10-15,NaN,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
1,10001,Electronics Technician / Item Master Controller,Ryzex Group,Bellingham,Washington,WA,2001-12-01,2012-04-01,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
2,10001,Machine Operator,comptec inc,Custer,Washington,WA,1997-01-01,1999-01-01,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
3,10003,maintenance technician,Winn residental,washington,District of Columbia,DC,NaN,NaN,"Necessary maintenance for ""Make Ready"" Plumbin...",10.0,False,2014-12-12 21:27:05 UTC,2014-12-12 21:27:05 UTC
4,10003,Electrical Helper,michael and son services,alexandria,Virginia,VA,NaN,NaN,repair and services of electrical construction,NaN,False,2014-12-12 21:27:05 UTC,2014-12-12 21:27:05 UTC


In [34]:
#Cleaning the text data
_clean_experience = experience[['Applicant.ID', 'Position.Name']]
_clean_experience['Position.Name'] = _clean_experience['Position.Name'].str.replace('[^a-zA-Z \n\.]',"")
_clean_experience['Position.Name'] = _clean_experience['Position.Name'].str.lower()
_clean_experience = _clean_experience.fillna(" ")
_clean_experience = _clean_experience.sort_values(by='Applicant.ID')
_clean_experience.head(20)



,Applicant.ID,Position.Name
2763,2,volunteer
2762,2,writer for the uloop blog
3759,3,marketing intern
3758,3,server
3757,3,prep cook
6277,6,project assistant
7490,8,deli clerkserver cashier food prep order taker
368,11,cashier
809,12,server
810,12,rec leader


In [35]:
#As the user experience is split across different rows, concatenating all rows into a single experience column

experience = _clean_experience.groupby('Applicant.ID', sort=False)['Position.Name'].apply(' '.join).reset_index()
experience.head(20)

,Applicant.ID,Position.Name
0,2,volunteer writer for the uloop blog
1,3,marketing intern server prep cook
2,6,project assistant
3,8,deli clerkserver cashier food prep order taker
4,11,cashier
5,12,server rec leader it program management intern
6,13,offica assistant cashier assistant
7,14,waitress host lifeguard
8,15,
9,18,baristacashier server receptionist


In [36]:
#Using Intereseted Positions data in a similar way to augment query

interests = pd.read_csv('Positions_Of_Interest.csv')
interests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6560 entries, 0 to 6559
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Applicant.ID          6560 non-null   int64 
 1   Position.Of.Interest  6558 non-null   object
 2   Created.At            6560 non-null   object
 3   Updated.At            6560 non-null   object
dtypes: int64(1), object(3)
memory usage: 205.1+ KB


In [37]:
_clean_interests = interests[['Applicant.ID', 'Position.Of.Interest']]
_clean_interests['Position.Of.Interest'] = _clean_interests['Position.Of.Interest'].str.replace('[^a-zA-Z \n\.]',"")
_clean_interests['Position.Of.Interest'] = _clean_interests['Position.Of.Interest'].str.lower()
_clean_interests = _clean_interests.fillna(" ")
_clean_interests = _clean_interests.sort_values(by='Applicant.ID')
_clean_interests.head(20)

,Applicant.ID,Position.Of.Interest
6437,96,server
1156,153,barista
1155,153,host
1154,153,server
1158,153,sales rep
1157,153,customer service rep
1952,256,host
1957,256,production area
1956,256,sales rep
1955,256,customer service rep


In [38]:
#As the user interest is split across different rows, concatenating all rows into a single interest column

interests = _clean_interests.groupby('Applicant.ID', sort=False)['Position.Of.Interest'].apply(' '.join).reset_index()
interests.head(20)

,Applicant.ID,Position.Of.Interest
0,96,server
1,153,barista host server sales rep customer service...
2,256,host production area sales rep customer servic...
3,438,customer service rep barista host server
4,568,receptionist customer service rep book keeper
5,601,customer service rep server line cook
6,867,driver customer service rep book keeper chef
7,938,host receptionist customer service rep server
8,1210,line cook host training community relations cu...
9,1251,fine arts retail parttime travel


In [39]:
#Merging the dataframes of user_view_history, experience and interests to form a query

#Here, merging user_view_history dataframe with experience dataframe taking outerjoin and replacing NAN by empty strings
_merge_user_view_experience = user_view_history.merge(experience, left_on='Applicant.ID', right_on='Applicant.ID', how='outer')
_merge_user_view_experience = _merge_user_view_experience.fillna(' ')
_merge_user_view_experience.head()


,Applicant.ID,View_Query,Position.Name
0,10000,cashiers valets needed wallypark newark,
1,10000,macys seasonal retail fragrance cashier gard...,
2,10001,part time showroom sales cashier grizzly ind...,machine operator account manager sales admini...
3,10002,event specialist part time advantage sales m...,
4,10002,bonefish kitchen staff bonefish grill green...,


In [40]:
#Merging the previous dataframe of user_view+experience with the interests dataframe
_merge_all = _merge_user_view_experience.merge(interests, left_on='Applicant.ID', right_on='Applicant.ID', how='outer')
_merge_all = _merge_all.fillna(" ")
_merge_all.head(20)

,Applicant.ID,View_Query,Position.Name,Position.Of.Interest
0,10000,cashiers valets needed wallypark newark,,
1,10000,macys seasonal retail fragrance cashier gard...,,
2,10001,part time showroom sales cashier grizzly ind...,machine operator account manager sales admini...,
3,10002,event specialist part time advantage sales m...,,
4,10002,bonefish kitchen staff bonefish grill green...,,
5,10003,entry level security officer securitas securi...,technician maintenance technician electrical ...,security officer
6,10004,pt teller chestereast th cleveland keybank ...,shift superviveur,
7,10006,housekeeper caregiver senior living san die...,,
8,10006,front desk coordinator for immediate opportuni...,,
9,10006,macys seasonal retail selling floor recovery e...,,


In [41]:
#combining all the columns to form the final user query

_merge_all["Query"] = _merge_all["View_Query"].map(str) + _merge_all["Position.Name"].map(str) +" "+ _merge_all["Position.Of.Interest"].map(str)

user_queries = _merge_all[['Applicant.ID', 'Query']]
user_queries.columns = ['User_Id', 'Query']
user_queries.head()

,User_Id,Query
0,10000,cashiers valets needed wallypark newark
1,10000,macys seasonal retail fragrance cashier gard...
2,10001,part time showroom sales cashier grizzly ind...
3,10002,event specialist part time advantage sales m...
4,10002,bonefish kitchen staff bonefish grill green...


In [43]:
user_queries.to_csv("user_queries.csv", index=True)

In [8]:
user_queries = pd.read_csv('user_queries.csv')

In [28]:
#Consider a random user with User_Id = user_id and check out recommended jobs to the user
random_user_id = 10006
index = np.where(user_queries['User_Id'] == random_user_id)[0][0]
random_user_query = user_queries.iloc[[index]]

In [29]:
#Using Cosine-similarity to calculate query-document matching for job recommendations

from sklearn.metrics.pairwise import cosine_similarity
random_user_vec = tfidf_vectorizer.transform(random_user_query['Query'])
random_user_job_sim = map(lambda x: cosine_similarity(random_user_vec, x), job_vecs)

In [31]:
#user_job_sim is a matrix of dimensions num_jobs x num_users and each row containing similarity score for each available job to a user
random_user_job_sim_list = list(random_user_job_sim)


In [35]:
#List top 10 jobs recommended to random_user

top_10_jobs = sorted(range(len(random_user_job_sim_list)), key=lambda i: random_user_job_sim_list[i], reverse=True)[:10]
print(top_10_jobs)
recommendation = pd.DataFrame(columns = ['User_Id', 'Job_Id'])
count = 0
for i in top_10_jobs:
    recommendation.at[count, 'User_Id'] = random_user_id
    recommendation.at[count,'Job_Id'] =  job_docs['Job_Id'][i]
    count += 1

recommendation

[408, 932, 4297, 933, 3956, 677, 2839, 1513, 3777, 3416]


,User_Id,Job_Id
0,10006,136563
1,10006,137172
2,10006,143661
3,10006,137174
4,10006,143224
5,10006,136853
6,10006,141530
7,10006,139832
8,10006,142619
9,10006,142156


In [41]:
#Adding Job description document to the recommended jobs

recommended_job_docs = recommendation.merge(job_docs, how='inner', on='Job_Id')
recommended_job_docs.drop(columns='Unnamed: 0', axis=1)

,User_Id,Job_Id,Job_Doc
0,10006,136563,medical receptionist needed asap officeteam sa...
1,10006,137172,massage therapy instructor premier education g...
2,10006,143661,registered nurse rn accepting applications tri...
3,10006,137174,management trainee intern port charlotte saras...
4,10006,143224,custodian janitorial porter host legoland disc...
5,10006,136853,scheduler officeteam chandler seasonal temp re...
6,10006,141530,cashier customer service p.c. richard son bric...
7,10006,139832,accounting intern celadon trucking services in...
8,10006,142619,health information mgr officeteam healthcare l...
9,10006,142156,payroll specialist accountemps lehi seasonal t...
